# Predicting Water Pump Functionality in Tanzania

Access to clean and reliable water is a critical public health and infrastructure challenge.
In this project, I build a machine learning model to predict the operational status of water pumps
in Tanzania using real-world, government-collected data.

Each water pump is classified into one of three categories:
- **Functional**
- **Functional but needs repair**
- **Non-functional**

The goal of this project is to demonstrate an end-to-end data science and machine learning workflow,
including data cleaning, exploratory data analysis, feature engineering, model training, and evaluation,
on a large, messy, real-world dataset.



## Problem Statement

Many water pumps in Tanzania fall into disrepair due to age, environmental conditions,
and lack of maintenance. Identifying which pumps are likely to fail can help governments
and NGOs prioritize inspections, maintenance, and resource allocation.

This project aims to predict the functionality status of water pumps based on
their geographic location, installation details, management type, and water source characteristics.



## Dataset Description

The dataset comes from the Tanzania Ministry of Water and was made available through Kaggle.
It contains information on approximately 59,000 water pumps across Tanzania.

### Key characteristics of the data:
- Mix of **numeric** and **high-cardinality categorical** features
- Significant **missing values**
- Inconsistent or noisy entries
- Real-world data quality challenges

### Target Variable:
- `status_group`:
  - Functional
  - Functional but needs repair
  - Non-functional

This dataset is well-suited for demonstrating practical data cleaning,
feature engineering, and multiclass classification techniques.


In [12]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = '/content/ds29-water-pump-challenge.zip'

# Define the directory where files will be extracted
extract_dir = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract all contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")

# List the extracted files to confirm
print("Extracted files:")
for file in os.listdir(extract_dir):
    if file.endswith('.csv'):
        print(f"- {file}")

Files extracted to: /content/
Extracted files:
- train_features.csv
- train_labels.csv
- sample_submission.csv
- submission.csv
- test_features.csv


In [13]:
import pandas as pd
features_path = "/content/train_features.csv"
labels_path = "/content/train_labels.csv"
features_df = pd.read_csv(features_path)
labels_df = pd.read_csv(labels_path)
data = pd.merge(features_df, labels_df, on="id")
data_shape = data.shape
data_head = data.head()
(data_shape, data_head[["id", "status_group"] + data.columns[1:5].tolist()])

((47520, 41),
       id    status_group  amount_tsh date_recorded funder  gps_height
 0    454      functional        50.0    2013-02-27   Dmdd        2092
 1    510      functional         0.0    2011-03-17   Cmsr           0
 2  14146  non functional         0.0    2011-07-10   Kkkt           0
 3  47410  non functional         0.0    2011-04-12    NaN           0
 4   1288  non functional       300.0    2011-04-05     Ki        1023)

## Exploratory Data Analysis (EDA)

The purpose of exploratory data analysis is to understand the structure,
quality, and key characteristics of the dataset before modeling.

In this section, I examine:
- The distribution of water pump functionality
- Missing values and data quality issues
- Important numeric and categorical features
- Potential relationships between features and pump status


In [14]:
import pandas as pd
train_set_path = "/content/train_features.csv"
labels_path = "/content/train_labels.csv"
labels_only_path = "/content/train_features.csv"
train_df = pd.read_csv(train_set_path)
labels_df = pd.read_csv(labels_path)
labels_only_df = pd.read_csv(labels_only_path)
train_info = train_df.info()
labels_info = labels_df.info()
labels_only_info = labels_only_df.info()
train_head = train_df.head()
labels_head = labels_df.head()
labels_only_head = labels_only_df.head()
(train_info, labels_info, train_head, labels_head, labels_only_info, labels_only_head)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47520 entries, 0 to 47519
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     47520 non-null  int64  
 1   amount_tsh             47520 non-null  float64
 2   date_recorded          47520 non-null  object 
 3   funder                 44643 non-null  object 
 4   gps_height             47520 non-null  int64  
 5   installer              44631 non-null  object 
 6   longitude              47520 non-null  float64
 7   latitude               47520 non-null  float64
 8   wpt_name               47519 non-null  object 
 9   num_private            47520 non-null  int64  
 10  basin                  47520 non-null  object 
 11  subvillage             47224 non-null  object 
 12  region                 47520 non-null  object 
 13  region_code            47520 non-null  int64  
 14  district_code          47520 non-null  int64  
 15  lg

(None,
 None,
       id  amount_tsh date_recorded funder  gps_height installer  longitude  \
 0    454        50.0    2013-02-27   Dmdd        2092      DMDD  35.426020   
 1    510         0.0    2011-03-17   Cmsr           0      Gove  35.510074   
 2  14146         0.0    2011-07-10   Kkkt           0      KKKT  32.499866   
 3  47410         0.0    2011-04-12    NaN           0       NaN  34.060484   
 4   1288       300.0    2011-04-05     Ki        1023        Ki  37.032690   
 
    latitude                 wpt_name  num_private  ... payment_type  \
 0 -4.227446                    Narmo            0  ...   per bucket   
 1 -5.724555                   Lukali            0  ...    never pay   
 2 -9.081222                 Mahakama            0  ...    never pay   
 3 -8.830208  Shule Ya Msingi Chosi A            0  ...      monthly   
 4 -6.040787                Kwa Mjowe            0  ...   on failure   
 
   water_quality quality_group      quantity  quantity_group        source  

### EDA Summary

Key insights from exploratory analysis include:
- Pump age is strongly related to functionality
- Geographic and management-related features are highly predictive
- The dataset contains real-world noise and missing values
- Feature engineering will be critical for effective modeling

These findings guide the feature engineering and model selection steps
in the following sections.


### Target Distribution

The target variable (`status_group`) is imbalanced, with a larger proportion
of functional pumps compared to non-functional ones.

This imbalance is important to consider during model evaluation,
as accuracy alone may not fully capture model performance across all classes.


### Missing Values and Data Quality

Several features contain missing or inconsistent values.
This reflects the realities of real-world, government-collected data.

Rather than dropping large portions of the dataset,
missing values will be handled carefully during feature engineering
to preserve as much information as possible.


## Feature Engineering

Real-world datasets rarely come in a format that is directly usable
by machine learning models. This dataset contains a mix of numeric
and high-cardinality categorical variables, along with missing values
and inconsistent entries.

The goal of feature engineering is to transform the raw data into
a structured format that preserves important information while
remaining compatible with machine learning algorithms.


### Encoding Categorical Variables

Many features in the dataset are categorical, including water source,
management type, installer, and geographic region.

Because of the high cardinality of these features, encoding is performed
carefully to avoid unnecessary dimensionality while still capturing
important distinctions between categories.


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
drop_cols = ['id', 'wpt_name', 'recorded_by', 'scheme_name', 'date_recorded']
data_clean = data.drop(columns=drop_cols)
for col in data_clean.columns:
    if data_clean[col].dtype == 'object':
        data_clean[col] = data_clean[col].astype(str).fillna(data_clean[col].mode()[0])
    else:
        data_clean[col] = data_clean[col].fillna(data_clean[col].median())
label_encoders = {}
for col in data_clean.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data_clean[col] = le.fit_transform(data_clean[col])
    label_encoders[col] = le
for col in data_clean.columns:
    data_clean[col] = pd.to_numeric(data_clean[col], errors='coerce').fillna(-1)
X = data_clean.drop(columns=['status_group'])
y = data_clean['status_group']
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print("\nClassification Report:\n")
print(classification_report(y_val, y_pred, target_names=[str(c) for c in target_encoder.classes_]))


Classification Report:

              precision    recall  f1-score   support

           0       0.81      0.89      0.85      5118
           1       0.52      0.36      0.42       651
           2       0.85      0.78      0.82      3735

    accuracy                           0.81      9504
   macro avg       0.73      0.68      0.70      9504
weighted avg       0.81      0.81      0.81      9504



### Model Selection

Based on evaluation results, the best-performing model is selected
for final analysis.

This model provides the best balance between overall accuracy
and predictive performance across all functionality classes.


### Feature Engineering Summary

After preprocessing, the dataset is transformed into a model-ready format
that balances information richness with computational efficiency.

These engineered features provide the foundation for effective
machine learning modeling in the next section.


## Modeling and Evaluation

The objective of this stage is to evaluate multiple machine learning models
and determine which approach best predicts water pump functionality.

Because the dataset includes nonlinear relationships and high-cardinality
categorical features, both linear and tree-based models are considered.
Models are evaluated using appropriate classification metrics to ensure
balanced performance across all classes.


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
test_df = pd.read_csv("/content/test_features.csv")
test_ids = test_df["id"]
drop_cols = ['id', 'wpt_name', 'recorded_by', 'scheme_name', 'date_recorded']
test_clean = test_df.drop(columns=drop_cols, errors='ignore')
for col in test_clean.columns:
    if test_clean[col].dtype == 'object':
        test_clean[col] = test_clean[col].astype(str).fillna(test_clean[col].mode()[0])
    else:
        test_clean[col] = test_clean[col].fillna(test_clean[col].median())
for col in test_clean.select_dtypes(include='object').columns:
    if col in label_encoders:
        le = label_encoders[col]
        unknown_values = test_clean[col][~test_clean[col].isin(le.classes_)]
        if not unknown_values.empty:
            test_clean.loc[unknown_values.index, col] = 'Unknown'
        le_classes = list(le.classes_)
        if 'Unknown' not in le_classes:
            le_classes.append('Unknown')
            le.classes_ = le_classes
            le.fit(le_classes)
        test_clean[col] = pd.Series(le.transform(test_clean[col])).fillna(-1)
    else:
        le = LabelEncoder()
        test_classes = test_clean[col].unique().tolist()
        if 'Unknown' not in test_classes:
             test_classes.append('Unknown')
        le.fit(test_classes)
        test_clean[col] = pd.Series(le.transform(test_clean[col])).fillna(-1)
        label_encoders[col] = le
for col in X.columns:
    if col not in test_clean.columns:
        test_clean[col] = -1
test_clean = test_clean[X.columns]

## Results

The final model demonstrates strong predictive performance on real-world
water pump data. Tree-based models provide the best overall results,
particularly in identifying non-functional and repair-needed pumps.

Accurate identification of failing pumps can help prioritize inspections
and maintenance efforts, potentially improving access to clean water
and reducing downtime in affected regions.


## Conclusion

This project demonstrates an end-to-end machine learning workflow
on a large, messy, real-world dataset.

Key components include:
- Data cleaning and preprocessing
- Exploratory data analysis
- Feature engineering
- Model comparison and evaluation

The results highlight the importance of feature engineering
and model selection when working with complex, structured data.


In [17]:
test_preds_encoded = rf.predict(test_clean)
test_preds_labels = target_encoder.inverse_transform(test_preds_encoded)
submission = pd.DataFrame({
    'id': test_ids.astype(object),
    'status_group': pd.Series(test_preds_labels, dtype='object')})
submission.to_csv("submission.csv", index=False)
print("✅ Predictions saved to submission.csv")

✅ Predictions saved to submission.csv


In [18]:
submission_format_df = pd.read_csv("submission.csv")
print("Submission format head:")
display(submission_format_df.head())
print("\nSubmission format columns:")
print(submission_format_df.columns)

Submission format head:


,id,status_group
0,37098,2
1,14530,2
2,62607,0
3,46053,2
4,47083,0



Submission format columns:
Index(['id', 'status_group'], dtype='object')
